# spaCy を用いた英語テキストの前処理

### ソフトウェアのインストール

spaCy
```
conda install spacy
python -m spacy download en_core_web_lg
```
- https://spacy.io/usage
- https://anaconda.org/anaconda/spacy

### 処理内容

- 前処理
- ベクトル化

### spaCy の日本語モデル

- 以下の spaCy の英語モデルのロードでエラーが出る場合はノートブックをリスタート

In [ ]:
import spacy

# 英語モデル
nlp = spacy.load('en_core_web_lg')

In [ ]:
import pandas as pd
import re

# フィードデータの読み込み
feeds = pd.read_csv('data/output_en.csv')

# title と summary を結合して text 列を作成
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='. ', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df

### 英語テキストに対する前処理

以下をまとめて行う関数 preprocess() を定義
- トークン化（単語に分割）
- 小文字化
- ストップワードの除去
- 見出し語化

（spaCyは、ステミング機能を提供していない）

In [ ]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
stop_pos = ['PUNCT', 'SPACE', 'SYM', 'X']
# うまく取り除けない単語や文字があれば追加
stop_words = []

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化 (1)

- Bag of Words (BoW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 例として最初の2行のテキストを処理
text_list = [df['text'].iloc[0], df['text'].iloc[1]]

# CountVectorizer
# - tokenizer=preprocess: トークン化処理に上で定義した preprocess を使用することを指定
vectorizer = CountVectorizer(tokenizer=preprocess)

# ベクトル化
vector = vectorizer.fit_transform(text_list)

In [ ]:
# 1行目
print(text_list[0])
print(vector[0])

In [ ]:
# ベクトルの単語との対応
for i in vector[0].indices:
    # print()
    # - end=' ': 改行の代わりに空白を出力
    print(vectorizer.get_feature_names_out()[i], end=' ')

In [ ]:
# 単語と頻度
[[vectorizer.get_feature_names_out()[i], vector[0, i]] for i in vector[0].indices]

In [ ]:
# 2行目
print(text_list[1])
print(vector[1])

In [ ]:
# ベクトルの単語との対応
for i in vector[1].indices:
    print(vectorizer.get_feature_names_out()[i], end=' ')

In [ ]:
# 単語と頻度
[[vectorizer.get_feature_names_out()[i], vector[1, i]] for i in vector[1].indices]

### テキストのベクトル化 (2)

- TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 例として最初の2行のテキストを処理
text_list = [df['text'].iloc[0], df['text'].iloc[1]]

# TfidfVectorizer
# - tokenizer=preprocess: トークン化処理に上で定義した preprocess を使用することを指定
vectorizer = TfidfVectorizer(tokenizer=preprocess)

# ベクトル化
vector = vectorizer.fit_transform(text_list)

In [ ]:
# 1行目
print(text_list[0])
print(vector[0])

In [ ]:
# ベクトルの単語との対応
for i in vector[0].indices:
    print(vectorizer.get_feature_names_out()[i], end=' ')

In [ ]:
# 単語とTF-IDF
[[vectorizer.get_feature_names_out()[i], vector[0, i]] for i in vector[0].indices]